In [1]:
import numpy as np
import pandas as pd

movies_data = pd.read_csv('imdb-movies-dataset.csv')

/Users/danielmarin/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [4]:
cleaned_data = movies_data.drop(['Poster', 'Description', 'Title', 'Metascore', 'Review Count', 'Review Title', 'Review'], axis=1)
cleaned_data['Votes'] = cleaned_data['Votes'].str.replace(',','').astype(float)

In [5]:
cleaned_data.fillna({
    'Year': cleaned_data['Year'].mean(),
    'Certificate': cleaned_data['Certificate'].mode()[0],
    'Duration (min)': cleaned_data['Duration (min)'].mean(),
    'Genre': cleaned_data['Genre'].mode()[0],
    'Rating': cleaned_data['Rating'].mean(),
    'Director': cleaned_data['Director'].mode()[0],
    'Cast': cleaned_data['Cast'].mode()[0],
    'Votes': cleaned_data['Votes'].mean(),
    }, inplace=True
)

In [7]:
cleaned_data['Genre'] = cleaned_data['Genre'].str.split(', ')
cleaned_data = cleaned_data.explode('Genre')

In [8]:
cleaned_data['encoded_Genre'] = cleaned_data.groupby('Genre')['Rating'].transform('mean')
cleaned_data['encoded_Duration'] = cleaned_data.groupby('Duration (min)')['Rating'].transform('mean')
cleaned_data['encoded_Certificate'] = cleaned_data.groupby('Certificate')['Rating'].transform('mean')
cleaned_data['encoded_Director'] = cleaned_data.groupby('Director')['Rating'].transform('mean')

In [9]:
cleaned_data.drop(['Certificate', 'Genre', 'Director', 'Cast', ], axis=1).corr()

,Year,Duration (min),Rating,Votes,encoded_Genre,encoded_Duration,encoded_Certificate,encoded_Director
Year,1.000000,0.000004,0.013386,0.029199,-0.001975,-0.002864,0.034108,0.010364
Duration (min),0.000004,1.000000,0.353358,0.231682,0.192738,0.838528,0.121130,0.348190
Rating,0.013386,0.353358,1.000000,0.379556,0.271443,0.421403,0.188594,0.846334
Votes,0.029199,0.231682,0.379556,1.000000,-0.025555,0.276866,0.319376,0.262779
encoded_Genre,-0.001975,0.192738,0.271443,-0.025555,1.000000,0.208084,-0.032485,0.253613
encoded_Duration,-0.002864,0.838528,0.421403,0.276866,0.208084,1.000000,0.146035,0.410345
encoded_Certificate,0.034108,0.121130,0.188594,0.319376,-0.032485,0.146035,1.000000,0.168256
encoded_Director,0.010364,0.348190,0.846334,0.262779,0.253613,0.410345,0.168256,1.000000


In [10]:
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [11]:
X = cleaned_data[['Votes', 'encoded_Certificate', 'encoded_Duration', 'encoded_Genre', 'encoded_Director']]
y = cleaned_data['Rating']

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.2)

In [16]:
rf = RandomForestRegressor()
rf.fit(X_train, y_train)

RandomForestRegressor()

In [17]:
rf_pred = rf.predict(X_test)
rf_mse = mean_squared_error(y_test, rf_pred)
rf_r2s = r2_score(y_test, rf_pred)
print(f"Mean squared error: {rf_mse}")
print(f"R2 score: {rf_r2s}")

Mean squared error: 0.10009030589549582
R2 score: 0.899914599009923


In [18]:
test_data = {'Certificate':'PG-13', 'Votes': [3458], 'Duration (min)': 172.0, 'Genre': 'Comedy', 'Director': 'David Winters'}

In [19]:
test_data["encoded_Certificate"] = cleaned_data[cleaned_data['Certificate'] == test_data['Certificate']]['Rating'].iloc[0]
test_data["encoded_Duration"] = cleaned_data[cleaned_data['Duration (min)'] == test_data['Duration (min)']]['Rating'].iloc[0]
test_data["encoded_Genre"] = cleaned_data[cleaned_data['Genre'] == test_data['Genre']]['Rating'].iloc[0]
test_data['encoded_Director'] = cleaned_data[cleaned_data['Director'] == test_data['Director']]['Rating'].iloc[0]

In [20]:
df_test = pd.DataFrame(test_data)
df_test.drop(['Certificate', 'Duration (min)', 'Genre', 'Director'], axis = 1, inplace = True)
df_test

,Votes,encoded_Certificate,encoded_Duration,encoded_Genre,encoded_Director
0,3458,7.3,8.2,6.4,6.5


In [23]:
predicted_rating_rf = rf.predict(df_test)
print(f"Predicted Rating: {predicted_rating_rf[0]}")

Predicted Rating: 7.3753860983743245
